#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter ITP Audit Dashboard ( 2020 ) Parameters
Dashboard that shows performance metrics across browser to see the impact of ITP.
 1. Follow the instructions from <a href="https://docs.google.com/document/d/1HaRCMaBBEo0tSKwnofWNtaPjlW0ORcVHVwIRabct4fY/edit?usp=sharing" target="_blank">this document</a>
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'auth_read': 'user',  # Credentials used for reading data.
  'auth_write': 'service',  # Credentials used for writing data.
  'recipe_name': '',  # Name of document to deploy to.
  'recipe_slug': 'ITP_Audit_Dashboard',  # BigQuery dataset for store dashboard tables.
  'cm_account_id': '',  # Campaign Manager Account Id.
  'cm_advertiser_ids': '',  # Optional: Comma delimited list of CM advertiser ids.
  'floodlight_configuration_id': '',  # Floodlight Configuration Id for the Campaign Manager floodlight report.
  'dv360_partner_ids': '',  # Comma delimited list of DV360 Partner ids.
  'dv360_advertiser_ids': '',  # Optional: Comma delimited list of DV360 Advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute ITP Audit Dashboard ( 2020 )
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'drive': {
      'auth': 'user',
      'hour': [
      ],
      'copy': {
        'destination': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'ITP Audit ','kind': 'string'}},
        'source': 'https://docs.google.com/spreadsheets/d/1rH_PGXOYW2mVdmAYnKbv6kcaB6lQihAyMsGtFfinnqg/'
      }
    }
  },
  {
    'dataset': {
      'auth': 'user',
      'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_ADVERTISER': {
            'values': {'field': {'description': 'Optional: Comma delimited list of DV360 Advertiser ids.','name': 'dv360_advertiser_ids','default': '','order': 6,'kind': 'integer_list'}}
          },
          'FILTER_PARTNER': {
            'values': {'field': {'description': 'Comma delimited list of DV360 Partner ids.','name': 'dv360_partner_ids','default': '','order': 5,'kind': 'integer_list'}}
          }
        },
        'body': {
          'params': {
            'type': 'TYPE_GENERAL',
            'groupBys': [
              'FILTER_ADVERTISER',
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_MEDIA_PLAN',
              'FILTER_INSERTION_ORDER',
              'FILTER_LINE_ITEM',
              'FILTER_PAGE_LAYOUT',
              'FILTER_WEEK',
              'FILTER_MONTH',
              'FILTER_YEAR',
              'FILTER_PARTNER',
              'FILTER_LINE_ITEM_TYPE',
              'FILTER_DEVICE_TYPE',
              'FILTER_BROWSER'
            ],
            'metrics': [
              'METRIC_MEDIA_COST_ADVERTISER',
              'METRIC_IMPRESSIONS',
              'METRIC_CLICKS',
              'METRIC_TOTAL_CONVERSIONS',
              'METRIC_LAST_CLICKS',
              'METRIC_LAST_IMPRESSIONS',
              'METRIC_CM_POST_CLICK_REVENUE',
              'METRIC_CM_POST_VIEW_REVENUE',
              'METRIC_REVENUE_ADVERTISER'
            ]
          },
          'timezoneCode': {'field': {'description': 'Timezone for report dates.','name': 'recipe_timezone','default': 'America/Los_Angeles','kind': 'timezone'}},
          'metadata': {
            'title': {'field': {'description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'prefix': 'ITP_Audit_Browser_','kind': 'string'}},
            'dataRange': 'LAST_365_DAYS',
            'format': 'CSV'
          }
        },
        'timeout': 90
      },
      'delete': False
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'account': {'field': {'description': 'Campaign Manager Account Id.','name': 'cm_account_id','default': '','order': 2,'kind': 'string'}},
        'body': {
          'type': 'FLOODLIGHT',
          'delivery': {
            'emailOwner': False
          },
          'floodlightCriteria': {
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_30_DAYS'
            },
            'floodlightConfigId': {
              'matchType': 'EXACT',
              'value': {'field': {'description': 'Floodlight Configuration Id for the Campaign Manager floodlight report.','name': 'floodlight_configuration_id','default': '','order': 4,'kind': 'integer'}},
              'dimensionName': 'dfa:floodlightConfigId',
              'kind': 'dfareporting#dimensionValue'
            },
            'metricNames': [
              'dfa:activityClickThroughConversions',
              'dfa:activityViewThroughConversions',
              'dfa:totalConversions',
              'dfa:totalConversionsRevenue'
            ],
            'reportProperties': {
              'includeUnattributedIPConversions': False,
              'includeUnattributedCookieConversions': True
            },
            'dimensions': [
              {
                'name': 'dfa:site',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:floodlightAttributionType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:interactionType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:pathType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:browserPlatform',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:platformType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:week',
                'kind': 'dfareporting#sortedDimension'
              }
            ]
          },
          'name': {'field': {'description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'prefix': 'ITP_Audit_Floodlight_','kind': 'string'}},
          'kind': 'dfareporting#report',
          'format': 'CSV',
          'schedule': {
            'active': True,
            'repeats': 'WEEKLY',
            'repeatsOnWeekDays': [
              'Sunday'
            ],
            'every': 1
          }
        },
        'timeout': 90
      },
      'delete': False,
      'out': {
        'bigquery': {
          'table': 'z_Floodlight_CM_Report',
          'is_incremental_load': False,
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      }
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'account': {'field': {'description': 'Campaign Manager Account Id.','name': 'cm_account_id','default': '','order': 2,'kind': 'string'}},
        'filters': {
          'dfa:advertiser': {
            'values': {'field': {'description': 'Optional: Comma delimited list of CM advertiser ids.','name': 'cm_advertiser_ids','default': '','order': 3,'kind': 'integer_list'}}
          }
        },
        'body': {
          'type': 'STANDARD',
          'delivery': {
            'emailOwner': False
          },
          'fileName': {'field': {'description': 'Name of the Campaign Manager browser report.','order': 1,'name': 'recipe_name','default': 'ITP_Audit_Dashboard_Browser','prefix': 'ITP_Audit_Browser_','kind': 'string'}},
          'format': 'CSV',
          'name': {'field': {'description': 'Name of the Campaign Manager browser report.','order': 1,'name': 'recipe_name','default': 'ITP_Audit_Dashboard_Browser','prefix': 'ITP_Audit_Browser_','kind': 'string'}},
          'kind': 'dfareporting#report',
          'criteria': {
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_365_DAYS'
            },
            'metricNames': [
              'dfa:impressions',
              'dfa:clicks',
              'dfa:totalConversions',
              'dfa:activityViewThroughConversions',
              'dfa:activityClickThroughConversions'
            ],
            'dimensionFilters': [
            ],
            'dimensions': [
              {
                'name': 'dfa:campaign',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:campaignId',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:site',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:advertiser',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:advertiserId',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:browserPlatform',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:platformType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:month',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:week',
                'kind': 'dfareporting#sortedDimension'
              }
            ]
          },
          'schedule': {
            'active': True,
            'repeats': 'WEEKLY',
            'repeatsOnWeekDays': [
              'Sunday'
            ],
            'every': 1
          }
        },
        'timeout': 90
      },
      'delete': False,
      'out': {
        'bigquery': {
          'table': 'z_CM_Browser_Report_Dirty',
          'is_incremental_load': False,
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      }
    }
  },
  {
    'sheets': {
      'tab': 'Enviroment',
      'sheet': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'ITP Audit ','kind': 'string'}},
      'header': True,
      'auth': 'user',
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Environment',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      },
      'range': 'A:B'
    }
  },
  {
    'sheets': {
      'tab': 'Browser',
      'sheet': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'ITP Audit ','kind': 'string'}},
      'header': True,
      'auth': 'user',
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Browser',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      },
      'range': 'A:C'
    }
  },
  {
    'sheets': {
      'tab': 'CM_Browser_lookup',
      'sheet': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'ITP Audit ','kind': 'string'}},
      'header': True,
      'auth': 'user',
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_CM_Browser_lookup',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      },
      'range': 'A:C'
    }
  },
  {
    'sheets': {
      'tab': 'Device_Type',
      'sheet': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'ITP Audit ','kind': 'string'}},
      'header': True,
      'auth': 'user',
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Device_Type',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      },
      'range': 'A:B'
    }
  },
  {
    'sheets': {
      'tab': 'Floodlight_Attribution',
      'sheet': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'ITP Audit ','kind': 'string'}},
      'header': True,
      'auth': 'user',
      'out': {
        'auth': 'user',
        'bigquery': {
          'table': 'z_Floodlight_Attribution',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      },
      'range': 'A:B'
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'name': {'field': {'description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'prefix': 'ITP_Audit_Browser_','kind': 'string'}}
      },
      'out': {
        'bigquery': {
          'table': 'z_Dv360_Browser_Report_Dirty',
          'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}}
        }
      }
    }
  },
  {
    'itp_audit': {
      'auth': 'user',
      'account': {'field': {'description': 'Campaign Manager Account Id.','name': 'cm_account_id','default': '','order': 2,'kind': 'string'}},
      'dataset': {'field': {'description': 'BigQuery dataset for store dashboard tables.','name': 'recipe_slug','default': 'ITP_Audit_Dashboard','order': 1,'kind': 'string'}},
      'timeout': 60,
      'sheet': {'field': {'description': 'Name of document to deploy to.','order': 1,'name': 'recipe_name','default': '','prefix': 'ITP Audit ','kind': 'string'}}
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
